In [ ]:
import os
import sys
sys.path.append("..")
from nbr.preparation import Preprocess, save_split, Corpus
from nbr.trainer import NBRTrainer
from nbr.model import UPCF
import torch
import random
import numpy as np
import optuna
import warnings
warnings.filterwarnings("ignore")

# TaFeng

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "ta_feng"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 32266, #items = 23812, #clicks = 817741 (#illegal records = 0)
After preprocessing: #users = 7358, #items = 11202, #clicks = 368951
Saving dataset in ./data//data_ta_feng/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 7358/7358 [00:17<00:00, 427.69it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:00<00:00, 173250.62it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:00<00:00, 152849.92it/s]


In [ ]:
def objective(trial):
    params = {
        "model": UPCF(
            user_num=corpus.n_users,
            item_num=corpus.n_items,
            recency=trial.suggest_int("recency", low=2, high=50, step=2),
            q=trial.suggest_categorical("q", [1, 5, 10, 50, 100]),
            alpha=trial.suggest_float("alpha", 0.1, 1.0, step=0.1),
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=25, high=500, step=25),
            corpus=corpus
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-07-04 22:25:28,746] A new study created in memory with name: no-name-4e6292cb-e3f9-467e-b17c-4d8c5ebe41b9


UPCF fitting...


100%|██████████| 7357/7357 [08:55<00:00, 13.74it/s]
[I 2023-07-04 22:34:31,443] Trial 0 finished with value: 0.10871439345788349 and parameters: {'recency': 40, 'q': 10, 'alpha': 0.2, 'nearest_neighbors_num': 400}. Best is trial 0 with value: 0.10871439345788349.


UPCF fitting...


100%|██████████| 7357/7357 [06:56<00:00, 17.66it/s]
[I 2023-07-04 22:41:33,525] Trial 1 finished with value: 0.1041300205858735 and parameters: {'recency': 10, 'q': 10, 'alpha': 0.9, 'nearest_neighbors_num': 325}. Best is trial 0 with value: 0.10871439345788349.


...

100%|██████████| 7357/7357 [04:11<00:00, 29.22it/s]
[I 2023-07-05 00:19:37,659] Trial 22 finished with value: 0.11025427338934153 and parameters: {'recency': 22, 'q': 5, 'alpha': 0.8, 'nearest_neighbors_num': 200}. Best is trial 18 with value: 0.11027248144869457.


UPCF fitting...


100%|██████████| 7357/7357 [04:16<00:00, 28.70it/s]
[I 2023-07-05 00:23:58,698] Trial 23 finished with value: 0.10949797224875923 and parameters: {'recency': 14, 'q': 5, 'alpha': 0.8, 'nearest_neighbors_num': 200}. Best is trial 18 with value: 0.11027248144869457.


UPCF fitting...


100%|██████████| 7357/7357 [03:53<00:00, 31.45it/s]
[I 2023-07-05 00:27:56,663] Trial 24 finished with value: 0.11015451005311834 and parameters: {'recency': 22, 'q': 5, 'alpha': 0.8, 'nearest_neighbors_num': 175}. Best is trial 18 with value: 0.11027248144869457.


Test:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)
params = {
    "model": UPCF(
        user_num=corpus.n_users,
        item_num=corpus.n_items,
        recency=study.best_params["recency"],
        q=study.best_params["q"],
        alpha=study.best_params["alpha"],
        nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
        corpus=corpus
    )
}

trainer.init_hyperparams(**params)

train dataset preparing...


100%|██████████| 7358/7358 [00:26<00:00, 275.67it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:02<00:00, 3628.40it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:02<00:00, 3159.63it/s]


UPCF fitting...


100%|██████████| 7358/7358 [00:01<00:00, 4743.39it/s]


In [ ]:
trainer.evaluate(mode="test")

100%|██████████| 7357/7357 [04:46<00:00, 25.64it/s]


{'precision': 0.05759140954193285,
 'recall': 0.13649849839913436,
 'ndcg': 0.11541800466459054}

# TaoBao

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 10 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "taobao"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(10, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 672404, #items = 638962, #clicks = 2015807 (#illegal records = 0)
After preprocessing: #users = 10092, #items = 22286, #clicks = 67991
Saving dataset in ./data//data_taobao/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 10092/10092 [00:48<00:00, 206.70it/s]


dev dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 199324.90it/s]


test dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 199846.35it/s]


In [ ]:
def objective(trial):
    params = {
        "model": UPCF(
            user_num=corpus.n_users,
            item_num=corpus.n_items,
            recency=trial.suggest_int("recency", low=2, high=50, step=2),
            q=trial.suggest_categorical("q", [1, 5, 10, 50, 100]),
            alpha=trial.suggest_float("alpha", 0.1, 1.0, step=0.1),
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=25, high=500, step=25),
            corpus=corpus
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-07-05 08:44:02,760] A new study created in memory with name: no-name-0b7c4001-aaac-4ca6-80f9-627555014dac


UPCF fitting...


100%|██████████| 9307/9307 [01:19<00:00, 116.55it/s]
[I 2023-07-05 08:45:28,448] Trial 0 finished with value: 0.07057254405320963 and parameters: {'recency': 40, 'q': 10, 'alpha': 0.2, 'nearest_neighbors_num': 400}. Best is trial 0 with value: 0.07057254405320963.


UPCF fitting...


100%|██████████| 9307/9307 [01:13<00:00, 126.98it/s]
[I 2023-07-05 08:46:47,639] Trial 1 finished with value: 0.07059692330307843 and parameters: {'recency': 10, 'q': 10, 'alpha': 0.9, 'nearest_neighbors_num': 325}. Best is trial 1 with value: 0.07059692330307843.


...

100%|██████████| 9307/9307 [01:16<00:00, 122.12it/s]
[I 2023-07-05 09:13:41,872] Trial 22 finished with value: 0.07200805588588075 and parameters: {'recency': 32, 'q': 50, 'alpha': 1.0, 'nearest_neighbors_num': 175}. Best is trial 11 with value: 0.07206451873009347.


UPCF fitting...


100%|██████████| 9307/9307 [01:19<00:00, 116.80it/s]
[I 2023-07-05 09:15:06,221] Trial 23 finished with value: 0.07142394402259548 and parameters: {'recency': 34, 'q': 50, 'alpha': 0.9, 'nearest_neighbors_num': 250}. Best is trial 11 with value: 0.07206451873009347.


UPCF fitting...


100%|██████████| 9307/9307 [01:10<00:00, 131.63it/s]
[I 2023-07-05 09:16:21,620] Trial 24 finished with value: 0.07079538003656198 and parameters: {'recency': 32, 'q': 5, 'alpha': 0.7000000000000001, 'nearest_neighbors_num': 75}. Best is trial 11 with value: 0.07206451873009347.


Test:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)
params = {
    "model": UPCF(
        user_num=corpus.n_users,
        item_num=corpus.n_items,
        recency=study.best_params["recency"],
        q=study.best_params["q"],
        alpha=study.best_params["alpha"],
        nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
        corpus=corpus
    )
}

trainer.init_hyperparams(**params)

train dataset preparing...


100%|██████████| 10092/10092 [00:57<00:00, 176.02it/s]


dev dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 25735.44it/s]


test dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 21703.57it/s]


UPCF fitting...


100%|██████████| 10092/10092 [00:01<00:00, 7572.77it/s]


In [ ]:
trainer.evaluate(mode="test")

100%|██████████| 9307/9307 [01:18<00:00, 118.54it/s]


{'precision': 0.008456000859568068,
 'recall': 0.08241108842806488,
 'ndcg': 0.05530549443807528}

# Dunnhumby

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "dunnhumby"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 2358/2358 [00:14<00:00, 162.48it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:00<00:00, 112280.65it/s]


test dataset preparing...


100%|██████████| 2357/2357 [00:00<00:00, 91785.81it/s]


In [ ]:
def objective(trial):
    params = {
        "model": UPCF(
            user_num=corpus.n_users,
            item_num=corpus.n_items,
            recency=trial.suggest_int("recency", low=2, high=50, step=2),
            q=trial.suggest_categorical("q", [1, 5, 10, 50, 100]),
            alpha=trial.suggest_float("alpha", 0.1, 1.0, step=0.1),
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=25, high=500, step=25),
            corpus=corpus
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-07-05 08:11:27,680] A new study created in memory with name: no-name-0455df30-ae52-4e6d-b678-388ff39b1803


UPCF fitting...


100%|██████████| 2357/2357 [01:03<00:00, 37.07it/s]
[I 2023-07-05 08:12:38,137] Trial 0 finished with value: 0.162296096705134 and parameters: {'recency': 40, 'q': 10, 'alpha': 0.2, 'nearest_neighbors_num': 400}. Best is trial 0 with value: 0.162296096705134.


UPCF fitting...


100%|██████████| 2357/2357 [00:48<00:00, 48.18it/s]
[I 2023-07-05 08:13:34,776] Trial 1 finished with value: 0.14460461844739012 and parameters: {'recency': 10, 'q': 10, 'alpha': 0.9, 'nearest_neighbors_num': 325}. Best is trial 0 with value: 0.162296096705134.


...

100%|██████████| 2357/2357 [00:40<00:00, 58.12it/s]
[I 2023-07-05 08:30:33,394] Trial 22 finished with value: 0.16422305668086676 and parameters: {'recency': 42, 'q': 5, 'alpha': 0.30000000000000004, 'nearest_neighbors_num': 200}. Best is trial 22 with value: 0.16422305668086676.


UPCF fitting...


100%|██████████| 2357/2357 [00:40<00:00, 58.59it/s]
[I 2023-07-05 08:31:20,477] Trial 23 finished with value: 0.16376822957414042 and parameters: {'recency': 44, 'q': 5, 'alpha': 0.30000000000000004, 'nearest_neighbors_num': 175}. Best is trial 22 with value: 0.16422305668086676.


UPCF fitting...


100%|██████████| 2357/2357 [00:30<00:00, 76.30it/s]
[I 2023-07-05 08:31:57,087] Trial 24 finished with value: 0.16340397647942972 and parameters: {'recency': 44, 'q': 5, 'alpha': 0.5, 'nearest_neighbors_num': 100}. Best is trial 22 with value: 0.16422305668086676.


Test:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)
params = {
    "model": UPCF(
        user_num=corpus.n_users,
        item_num=corpus.n_items,
        recency=study.best_params["recency"],
        q=study.best_params["q"],
        alpha=study.best_params["alpha"],
        nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
        corpus=corpus
    )
}

trainer.init_hyperparams(**params)

In [ ]:
trainer.evaluate(mode="test")

100%|██████████| 2357/2357 [00:42<00:00, 55.77it/s]


{'precision': 0.11671616461603734,
 'recall': 0.1662591039440097,
 'ndcg': 0.15995339543947557}